## Checkers Jump
1. detect two objects of different colors in the camera frame.
2. from seminar_6 we get our camera intrinsic matrix A and distortion coefficients (dis) 
3. Calculate the position the first object must move to in order to "capture" the second object (i.e., jump over it), 
and draw the target position on the image.

In [1]:
# detect two objects of different colors in HSV

import cv2
import numpy as np

# Callback function (does nothing)
def nothing(x):
    pass

# Open webcam
cap = cv2.VideoCapture(0)

# Create a window
cv2.namedWindow("mask")

# Create trackbars to adjust the HSV threshold values
cv2.createTrackbar("low_H", "mask", 0, 255, nothing)  # Hue (lower limit)
cv2.createTrackbar("low_S", "mask", 0, 255, nothing)  # Saturation (lower limit)
cv2.createTrackbar("low_V", "mask", 0, 255, nothing)  # Value (lower limit)
cv2.createTrackbar("high_H", "mask", 155, 255, nothing)  # Hue (upper limit)
cv2.createTrackbar("high_S", "mask", 155, 255, nothing)  # Saturation (upper limit)
cv2.createTrackbar("high_V", "mask", 155, 255, nothing)  # Value (upper limit)

while True:
    # Capture a frame from the webcam
    success, frame = cap.read()
    if not success:
        break

    # Convert the frame from BGR to HSVя
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the trackbar values for HSV range
    low_H = cv2.getTrackbarPos("low_H", "mask")
    low_S = cv2.getTrackbarPos("low_S", "mask")
    low_V = cv2.getTrackbarPos("low_V", "mask")
    high_H = cv2.getTrackbarPos("high_H", "mask")
    high_S = cv2.getTrackbarPos("high_S", "mask")
    high_V = cv2.getTrackbarPos("high_V", "mask")

    # Create the lower and upper bounds for the HSV range
    lower_bound = np.array([low_H, low_S, low_V])
    upper_bound = np.array([high_H, high_S, high_V])

    # Create the mask to filter the colors within the specified HSV range
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Display the original, mask, and filtered result
    cv2.imshow("Original", frame)
    cv2.imshow("mask", mask)

    # Press 'q' to exit the loop
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(30)



-1

In [2]:
'''
Calculate the position the first object must move to in order to "capture" the second object (i.e., jump over it), 
and draw the target position on the image.
'''

import numpy as np
import cv2
import math

# Camera intrinsic matrix
A = np.array([[653.79017244, 0.00000000, 325.45730598],
              [0.00000000, 654.02497422, 240.77670025],
              [0.00000000, 0.00000000, 1.00000000]])

# Camera distortion coefficients
dist = np.array([[-0.19878638,  0.50415653,  0.00260482,  0.00095636,  -0.52617064]])

# Function to convert image coordinates to real-world coordinates
def pic2r(A, alpha, beta, x, y, h):
    fx = A[0, 0]
    fy = A[1, 1]
    cx = A[0, 2]
    cy = A[1, 2]

    Xc = (x - cx) / fx
    Yc = (y - cy) / fy

    Zr = h / math.tan(beta)

    Xr = Zr * Xc
    Yr = Zr * Yc

    return Xr, Yr

cam = cv2.VideoCapture(0)

while True:
    success, frame = cam.read()
    if not success:
        print("Cannot read frame. Exiting")
        break

     # Apply blurring and convert frame to HSV color space
    blurred = cv2.blur(frame, (7, 7))
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    #  Detect red object
    mask1 = cv2.inRange(hsv, (0, 150, 143), (219, 255, 255)) # first obj - red
    mask2 = cv2.inRange(hsv, (48, 117, 16), (92, 255, 166))  # second obj - green

    # Combine both masks
    combined_mask = cv2.bitwise_or(mask1, mask2)
    cv2.imshow("Mask", combined_mask)

    # Function to detect objects based on color mask
    def detect_object(mask, color):
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
        
        num_labels = output[0]
        stats = output[2]
        objects = []
        
        for i in range(1, num_labels):
            a = stats[i, cv2.CC_STAT_AREA]
            t = stats[i, cv2.CC_STAT_TOP]
            l = stats[i, cv2.CC_STAT_LEFT]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            
            if a >= 1500:
                cx = l + w // 2  
                cy = t + h // 2  
                objects.append((cx, cy))
                cv2.rectangle(frame, (l, t), (l + w, t + h), color, 3)
        
        return objects

    objects1 = detect_object(mask1, (0, 0, 0))   # Màu đỏ
    objects2 = detect_object(mask2, (0, 0, 0))   # Màu xanh lá

    # Display object positions
    if objects1:
        obj1 = objects1[0]  
        cv2.putText(frame, "Obj1", (obj1[0] - 20, obj1[1] - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)

    if objects2:
        obj2 = objects2[0]  
        cv2.putText(frame, "Obj2", (obj2[0] - 20, obj2[1] - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
        
    # Calculate target position if both objects are detected
    if objects1 and objects2:
        obj1 = objects1[0]  
        obj2 = objects2[0]  
        
        target_x = 2 * obj2[0] - obj1[0] 
        target_y = 2 * obj2[1] - obj1[1]
        
        cv2.circle(frame, (target_x, target_y), 10, (255, 0, 0), -1)
        
        cv2.putText(frame, "TARGET", (target_x - 40, target_y - 40), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0, 0, 255), 1, cv2.LINE_AA)
        '''
        (target_x - 40, target_y - 40) → The position (x, y) where the text will appear - moves the text slightly left and upward 
        1 → Font scale (the size of the text); (0, 0, 255) → The color of the text in BGR format; 1 → Thickness of the text stroke.

        '''
        # Convert to real-world coordinates
        xr, yr = pic2r(A, 0, (90 - 54) * math.pi / 180, target_x, target_y, 30)  # beta = (90 - 54) * math.pi / 180 () represents the angle relative to the horizontal plane.
        cv2.putText(frame, f"Real: ({xr:.2f}, {yr:.2f})", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 3)
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(30) & 0xFF
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()



## Key point
implement object search (keypoints) from template(detail) on video

In [4]:
'''
The code still fails to 'accurately identify matching keypoints between the reference image and the current frame.
As observed, many keypoints do not belong to the object (these from the hand) but are mistakenly identified. Is there a way to optimize this further?
'''

import numpy as np
import cv2

# Path to reference image and video file
ref_path = "ref.jpg"
vid_path = "vid.mp4"

reference = cv2.imread(ref_path, cv2.IMREAD_GRAYSCALE)
cam = cv2.VideoCapture(vid_path)

# Инициализация SIFT , # Initialize SIFT (Scale-Invariant Feature Transform) detector
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(reference, None)

# FLANN-поиск # FLANN (Fast Library for Approximate Nearest Neighbors) matcher settings
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=100)  # Number of checks for nearest neighbors
flann = cv2.FlannBasedMatcher(index_params, search_params)

while True:
    ret, frame = cam.read()
    
    if not ret:
        cam.release()
        cam = cv2.VideoCapture(vid_path)
        continue
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    keypoints2, descriptors2 = sift.detectAndCompute(gray_frame, None)

    # Perform feature matching only if there are valid descriptors
    if descriptors2 is not None and len(descriptors2) > 0:
        matches = flann.knnMatch(descriptors1, descriptors2, k=2)
        
        good_matches = []
        for m, n in matches:
            if m.distance < 0.65 * n.distance:
                good_matches.append(m)
        
        result = cv2.drawMatches(reference, keypoints1, frame, keypoints2, good_matches[:30], None,
                                 flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        result_resized = cv2.resize(result, (1000, 900))
    else:
        result = frame
    
    cv2.imshow("matched", result_resized)
    
    key = cv2.waitKey(5000) & 0xFF
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

-1